In [165]:
import networkx as nx
import numpy as np
import pandas as pd

In [178]:
paper_to_paper = np.load("test_data_label_propagation/edgelist_cites_test_label_propagation.npy")
papers = np.load("test_data_label_propagation/paper_array_test_label_propagation.npy")
labels = np.load("test_data_label_propagation/paper_label_test_label_propagation.npy")

edgelist_reindexed = []
papers_reindexed = []
mapping = {}
for i in range(len(papers)):
    mapping[papers[i]] = i
    papers_reindexed.append(i)

for i in range(len(paper_to_paper.T)):
    edge = paper_to_paper[:,i]
    edgelist_reindexed.append([mapping[edge[0]],mapping[edge[1]]])

cites = np.array(edgelist_reindexed).T
papers = papers_reindexed

In [179]:
np.unique(labels[~np.isnan(labels)],return_counts = True)

(array([  1.,  17.,  23.,  24.,  97., 139.]), array([4, 3, 6, 1, 1, 1]))

In [168]:
label_dict = {}
for i in range(len(papers)):
    label_dict[papers[i]] = labels[i]

G = nx.Graph()
for i in range(len(papers)):
    G.add_node(papers[i])
nx.set_node_attributes(G,label_dict,name="label")

edgelist = []
for edge in cites.T:
    edgelist.append((edge[0],edge[1]))
G.add_edges_from(edgelist)
print(len(G))

#print(len(G))


#print(G.number_of_edges())
#print(G.number_of_nodes())

11530


In [169]:
len(max(nx.connected_components(G), key=len))

11516

In [170]:
len(G)

11530

In [171]:
#communities = nx.algorithms.community.label_propagation.asyn_lpa_communities(G)


In [172]:
#communities_list = list(communities)
#print(len(communities_list))

In [173]:
#required_number_of_labels = len(max(nx.connected_components(G), key=len))
attrs_dict = nx.get_node_attributes(G,"label")
labels_propagated = labels
#while True:
for i in range(5):
    for node in G.nodes:
        neighbor_labels = {}
        neighbors = list(G.neighbors(node))
        for i in neighbors:
            if not np.isnan(attrs_dict[i]):
                if attrs_dict[i] in neighbor_labels:
                    neighbor_labels[attrs_dict[i]] = neighbor_labels[attrs_dict[i]] + 1
                else:
                    neighbor_labels[attrs_dict[i]] = 1
            # if dict contains a value
            if neighbor_labels:
                prop_label = np.random.choice([key for key in neighbor_labels.keys() 
                                                if neighbor_labels[key]==max(neighbor_labels.values())])
                G.nodes[node]["label"] = prop_label
                labels_propagated[node] = prop_label
                attrs_dict[node] = prop_label
    #if required_number_of_labels == sum(~np.isnan(labels_propagated)):
    #    break

In [174]:
sum(~np.isnan(labels_propagated))

11516

In [175]:
label_number = []
for i in labels_propagated:
    if not np.isnan(i):
        label_number.append(i)
len(label_number)

11516

In [176]:
(np.unique(labels_propagated, return_counts = True))

(array([  1.,  17.,  23., 139.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan]),
 array([3934,  145, 7388,   49,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1]))